In [1]:
import pandas as pd
import numpy as np
import os
import sys
import warnings
from multiprocessing import Pool
from sklearn.metrics import confusion_matrix
import seaborn as sn
from pandas.io.parsers.readers import read_csv

from multiprocessing import Pool
from pdb import set_trace
from scipy.stats import spearmanr, pearsonr
from scipy.spatial.distance import cdist
import time
import matplotlib.pyplot as plt

In [ ]:
# train data for similarity
train_binary_df = pd.DataFrame()
train_true_df = pd.DataFrame()
for idx in selected_user:
    train_summary = pd.read_csv("/home/grigriko/seed5_1st1e-4/2nd_lr_1e-05/pred/pred_df_output_" + str(idx) + ".csv")
    train_summary['pred_by_th'] = 0
    best_th = selected_df.loc[selected_df['id']==int(idx)]['th']#ユーザーごとにyoudensJが最大の時のthを求める

    best_th = best_th.iloc[-1].strip('[]')
    best_th = float(best_th)#float型にして不等号を使える形に
    for i in range(len(train_summary)):
        if train_summary.loc[i, 'pred_vec'] > best_th:#予測値がthより大きければpred_by_thを1に変更
            train_summary.loc[i, 'pred_by_th'] = 1
    train_binary_df[idx] = train_summary['pred_by_th']
    train_true_df[idx] = train_summary['test_vec']

In [ ]:
# train data all user
all_binary_train = pd.DataFrame()
all_true_train = pd.DataFrame()
for idx in test_best_youdensJ['id']:
    all_train_summary = pd.read_csv("/home/grigriko/seed5_1st1e-4/2nd_lr_1e-05/pred/pred_df_output_" + str(idx) + ".csv")
    all_train_summary['pred_by_th'] = 0#新たな列を宣言し、全て0で埋めておく
    best_th = test_best_youdensJ.loc[test_best_youdensJ['id']==int(idx)]['th']#ユーザーごとにyoudensJが最大の時のthを求める
    best_th = best_th.iloc[-1].strip('[]')
    best_th = float(best_th)#float型にして不等号を使える形に
    for i in range(len(all_train_summary)):
        if all_train_summary.loc[i, 'pred_vec'] > best_th:#予測値がthより大きければpred_by_thを1に変更
            all_train_summary.loc[i, 'pred_by_th'] = 1
    all_binary_train[idx] = all_train_summary['pred_by_th']
    all_true_train[idx] = all_train_summary['test_vec']
all_binary_train    

In [ ]:
rest_binary_train = all_binary_train.drop(train_binary_df, axis=1)
rest_true_train = all_true_train.drop(train_true_df, axis=1)
rest_true_train

In [ ]:
# valid data for similarity
iANN_ppsed_pred_binary_df = pd.DataFrame()
iANN_ppsed_pred_continuous_df = pd.DataFrame()
iANN_ppsed_true_df = pd.DataFrame()
for idx in selected_user:
    
    iANN_ppsed_summary = pd.read_csv("/home/grigriko/seed5_1st1e-4/2nd_lr_1e-05/pred/valid_pred_df_output_" + str(idx) + ".csv")
    iANN_ppsed_summary['pred_by_th'] = 0#新たな列を宣言し、全て0で埋めておく
    best_th = selected_df.loc[selected_df['id']==int(idx)]['th']#ユーザーごとにyoudensJが最大の時のthを求める

    best_th = best_th.iloc[-1].strip('[]')
    best_th = float(best_th)#float型にして不等号を使える形に
    for i in range(len(iANN_ppsed_summary)):
        if iANN_ppsed_summary.loc[i, 'pred_vec'] > best_th:#予測値がthより大きければpred_by_thを1に変更
            iANN_ppsed_summary.loc[i, 'pred_by_th'] = 1
    
    iANN_ppsed_pred_binary_df[idx] = iANN_ppsed_summary['pred_by_th']#thresholdに基づいた0,1の予測値
    iANN_ppsed_pred_continuous_df[idx] = iANN_ppsed_summary['pred_vec']#iANNが出力した予測値そのまま
    iANN_ppsed_true_df[idx] = iANN_ppsed_summary['test_vec']#実際レビューしたかどうかの教師データ

In [ ]:
# valid data all user
all_binary_df = pd.DataFrame()
all_true_df = pd.DataFrame()
for idx in test_best_youdensJ['id']:
    all_summary = pd.read_csv("/home/grigriko/seed5_1st1e-4/2nd_lr_1e-05/pred/valid_pred_df_output_" + str(idx) + ".csv")
    all_summary['pred_by_th'] = 0#新たな列を宣言し、全て0で埋めておく
    best_th = test_best_youdensJ.loc[test_best_youdensJ['id']==int(idx)]['th']#ユーザーごとにyoudensJが最大の時のthを求める
    best_th = best_th.iloc[-1].strip('[]')
    best_th = float(best_th)#float型にして不等号を使える形に
    for i in range(len(all_summary)):
        if all_summary.loc[i, 'pred_vec'] > best_th:#予測値がthより大きければpred_by_thを1に変更
            all_summary.loc[i, 'pred_by_th'] = 1
    all_binary_df[idx] = all_summary['pred_by_th']
    all_true_df[idx] = all_summary['test_vec']

In [ ]:
rest_binary_df = all_binary_df.drop(iANN_ppsed_pred_binary_df, axis=1)
rest_true_df = all_true_df.drop(iANN_ppsed_true_df, axis=1)
rest_true_df

In [ ]:
# test data for prediction
test_binary_df = pd.DataFrame()
test_continuous_df = pd.DataFrame()
test_true_df = pd.DataFrame()
for idx in selected_user:
    iANN_ppsed_summary = pd.read_csv("/home/grigriko/seed5_1st1e-4/2nd_lr_1e-05/test/pred_df_output" + str(idx) + ".csv")
    iANN_ppsed_summary['pred_by_th'] = 0#新たな列を宣言し、全て0で埋めておく
    best_th = selected_df.loc[selected_df['id']==int(idx)]['th']#ユーザーごとにyoudensJが最大の時のthを求める

    best_th = best_th.iloc[-1].strip('[]')
    best_th = float(best_th)#float型にして不等号を使える形に
    for i in range(len(iANN_ppsed_summary)):
        if iANN_ppsed_summary.loc[i, 'pred_vec'] > best_th:#予測値がthより大きければpred_by_thを1に変更
            iANN_ppsed_summary.loc[i, 'pred_by_th'] = 1
    
    test_binary_df[idx] = iANN_ppsed_summary['pred_by_th']#thresholdに基づいた0,1の予測値
    test_continuous_df[idx] = iANN_ppsed_summary['pred_vec']#iANNが出力した予測値そのまま
    test_true_df[idx] = iANN_ppsed_summary['test_vec']#実際レビューしたかどうかの教師データ

In [ ]:
# test data all user
all_binary_test = pd.DataFrame()
all_true_test = pd.DataFrame()
for idx in test_best_youdensJ['id']:
    all_summary = pd.read_csv("/home/grigriko/seed5_1st1e-4/2nd_lr_1e-05/test/pred_df_output" + str(idx) + ".csv")
    all_summary['pred_by_th'] = 0#新たな列を宣言し、全て0で埋めておく
    best_th = test_best_youdensJ.loc[test_best_youdensJ['id']==int(idx)]['th']#ユーザーごとにyoudensJが最大の時のthを求める
    best_th = best_th.iloc[-1].strip('[]')
    best_th = float(best_th)#float型にして不等号を使える形に
    for i in range(len(all_summary)):
        if all_summary.loc[i, 'pred_vec'] > best_th:#予測値がthより大きければpred_by_thを1に変更
            all_summary.loc[i, 'pred_by_th'] = 1
    all_binary_test[idx] = all_summary['pred_by_th']
    all_true_test[idx] = all_summary['test_vec']

In [ ]:
rest_binary_test = all_binary_test.drop(test_binary_df, axis=1)
rest_true_test = all_true_test.drop(test_true_df, axis=1)
rest_true_test

In [ ]:
# trainとvalidが同じ名前で保存されている
train_binary_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/binary.csv",index=False)
train_true_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/true.csv",index=False)
all_binary_train.to_csv("/home/grigriko/collaborative_filtering/preprocessing/all_binary_df.csv", index=False)
all_true_train.to_csv("/home/grigriko/collaborative_filtering/preprocessing/all_user_true.csv", index=False)
rest_binary_train.to_csv("/home/grigriko/collaborative_filtering/preprocessing/rest_binary.csv", index=False)
rest_true_train.to_csv("/home/grigriko/collaborative_filtering/preprocessing/rest_true.csv", index=False)

iANN_ppsed_pred_binary_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/binary.csv",index=False)
iANN_ppsed_pred_continuous_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/continuous.csv",index=False)
iANN_ppsed_true_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/true.csv",index=False)
all_binary_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/all_binary_df.csv", index=False)
all_true_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/all_user_true.csv", index=False)
rest_binary_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/rest_binary.csv", index=False)
rest_true_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/rest_true.csv", index=False)

test_binary_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/test_binary.csv",index=False)
test_continuous_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/test_continuous.csv",index=False)
test_true_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/test_true.csv",index=False)
all_binary_test.to_csv("/home/grigriko/collaborative_filtering/preprocessing/all_binary_test.csv",index=False)
all_true_test.to_csv("/home/grigriko/collaborative_filtering/preprocessing/all_true_test.csv",index=False)
rest_binary_test.to_csv("/home/grigriko/collaborative_filtering/preprocessing/rest_binary_test.csv",index=False)
rest_true_test.to_csv("/home/grigriko/collaborative_filtering/preprocessing/rest_true_test.csv",index=False)

In [ ]:
integrate_df = pd.concat([iANN_ppsed_pred_binary_df, rest_true_df], axis=1)
integrate_df.to_csv("/home/grigriko/collaborative_filtering/preprocessing/integrate_df.csv")